In [1]:
# for part 4.
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten
from keras.layers import Dropout, Input, BatchNormalization, Reshape, Lambda
from keras.layers import LSTM, Bidirectional, TimeDistributed, merge
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K

import numpy as np
from skimage import io as skimio
from skimage import color as skimcolor

import os
import csv
import time

Using TensorFlow backend.


In [2]:
csv_files_train = "./data/train.csv"
csv_files_eval = "./data/valid.csv"
n_epochs = 1
gpu = "" # help="GPU 0,1 or '' ", default=''

train_batch_size=64
eval_batch_size=64
input_shape=(117, 1669, 1)
# optimizer=SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5) # Darryl's courses way
optimizer=SGD(lr=1e-3, decay=0.95, momentum=0.9, nesterov=True, clipnorm=5) # from previous code
alphabet=" !\"#&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz|~"
alphabet_codes = list(range(len(alphabet)))
n_classes = len(alphabet)
csv_delimiter='\t'
keep_prob_dropout = 0.7

save_model_file = "model1.h5"
pickle_file = "model1.pkl"

if not os.path.exists('results'):
        os.makedirs('results')

## Create data generator

In [3]:
char_map_str = """
<SPACE> 0
! 1
" 2
# 3
& 4
' 5
( 6
) 7
* 8
+ 9
, 10
- 11
. 12
/ 13
0 14
1 15
2 16
3 17
4 18
5 19
6 20
7 21
8 22
9 23
: 24
; 25
< 26
= 27
> 28
? 29
A 30
B 31
C 32
D 33
E 34
F 35
G 36
H 37
I 38
J 39
K 40
L 41
M 42
N 43
O 44
P 45
Q 46
R 47
S 48
T 49
U 50
V 51
W 52
X 53
Y 54
[ 55
] 56
_ 57
a 58
b 59
c 60
d 61
e 62
f 63
g 64
h 65
i 66
j 67
k 68
l 69
m 70
n 71
o 72
p 73
q 74
r 75
s 76
t 77
u 78
v 79
w 80
x 81
y 82
z 83
| 84
~ 85
"""
# the "blank" character is mapped to 28

char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)+1] = ch
index_map[2] = ' '

In [4]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        else:
            ch = char_map[c]
        int_sequence.append(ch)
    return int_sequence

def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    text = []
    for c in int_sequence:
        ch = index_map[c]
        text.append(ch)
    return text

In [5]:
# import training data
with open(csv_files_train) as f:
    readr = csv.reader(f, delimiter="\t")
    train = [row for row in readr]

tr_imgs = np.array([skimio.imread(r[0]) for r in train])
tr_imgs = np.expand_dims(tr_imgs, 3)
tr_len = len(tr_imgs)

max_string_length = max([len(r[1]) for r in train])
tr_labs = np.ones((tr_len, max_string_length)) * 86
tr_label_len = np.zeros((tr_len, 1))
for i in range(tr_len):
    label = np.array(text_to_int_sequence(train[i][1])) 
    tr_labs[i, :len(label)] = label
    tr_label_len[i] = len(label)

In [6]:
# import validation data
with open(csv_files_eval) as f:
    readr = csv.reader(f, delimiter="\t")
    valid = [row for row in readr]

val_imgs = np.array([skimio.imread(r[0]) for r in valid])
val_imgs = np.expand_dims(val_imgs, 3)
val_len = len(val_imgs)

# max_string_length = max([len(r[1]) for r in valid])
val_labs = np.ones((val_len, max_string_length)) * 86
val_label_len = np.zeros((val_len, 1))
for i in range(val_len):
    label = np.array(text_to_int_sequence(valid[i][1])) 
    val_labs[i, :len(label)] = label
    val_label_len[i] = len(label)

# Start making the model

In [7]:
inputs = Input(shape=input_shape)

## The deep cnn part

In [8]:
# conv1 - maxPool2x2
conv1 = Conv2D(64, (3,3), activation="relu", input_shape=input_shape,
                 padding="same")(inputs)
conv1 = MaxPooling2D(pool_size=(2, 2), padding="same")(conv1)

# conv2 - maxPool2x2
conv2 = Conv2D(128, (3,3), activation="relu", input_shape=input_shape,
                 padding="same")(conv1)
conv2 = MaxPooling2D(pool_size=(2, 2), padding="same")(conv2)

# conv3 - w/batch-norm
conv3 = Conv2D(256, (3,3), input_shape=input_shape, padding="same")(conv2)
conv3 = BatchNormalization(axis=-1)(conv3)
conv3 = Activation("relu")(conv3)

# conv4 - maxPool 2x1
conv4 = Conv2D(256, (3,3), activation="relu", input_shape=input_shape,
                 padding="same")(conv3)
conv4 = MaxPooling2D(pool_size=(2, 1), padding="same")(conv4)

# conv5 - w/batch-norm
conv5 = Conv2D(512, (3,3), input_shape=input_shape, padding="same")(conv4)
conv5 = BatchNormalization(axis=-1)(conv5)
conv5 = Activation("relu")(conv5)

# conv6 - maxPool 2x1
conv6 = Conv2D(512, (3,3), activation="relu", input_shape=input_shape,
                 padding="same")(conv5)
conv6 = MaxPooling2D(pool_size=(2, 1), padding="same")(conv6)

# conv 7 - w/batch-norm
conv7 = Conv2D(512, (2,2), input_shape=input_shape, padding="valid")(conv6)
conv7 = BatchNormalization(axis=-1)(conv7)
conv7 = Activation("relu")(conv7)

# reshape output
# from [batch, height, width, features]
# to [batch, width, height x features]
shp = Model(inputs=[inputs], outputs=[conv7]).output_shape
conv_out = Reshape((shp[2], shp[1]*shp[3]))(conv7)

## Bidirectional LSTM

In [9]:
bi1 = Bidirectional(LSTM(256, return_sequences=True))(conv_out)
bi2 = Bidirectional(LSTM(256, return_sequences=True))(bi1)

bi_out = Dropout(keep_prob_dropout)(bi2)
bi_out = TimeDistributed(Dense(n_classes))(bi_out)

soft = Activation("softmax")(bi_out)

## CTC loss

In [ ]:
model1 = Model(inputs=inputs, outputs=soft)
model1.output_length = lambda x: x
# print(model1.summary())

In [ ]:
# Darryl's stuff

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def add_ctc_loss(input_to_softmax):
    the_labels = Input(name='the_labels', shape=(max_string_length), dtype='float32')
    input_lengths = Input(name='input_length', shape=(1,), dtype='int64')
    label_lengths = Input(name='label_length', shape=(1,), dtype='int64')
    output_lengths = Lambda(input_to_softmax.output_length)(input_lengths)
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')(
        [input_to_softmax.output, the_labels, output_lengths, label_lengths])
    model = Model(
        inputs=[input_to_softmax.input, the_labels, input_lengths, label_lengths], 
        outputs=loss_out)
    return model


model = add_ctc_loss(model1)

# use a dummy lambda function for the loss b/c CTC loss is built into model
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)

In [10]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [13]:
labels = Input(name='the_labels', shape=[max_string_length], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([soft, labels, input_length, label_length])

In [16]:
model = Model(inputs=[inputs, labels, input_length, label_length],
             outputs=loss_out)
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 117, 1669, 1)  0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 117, 1669, 64) 640         input_1[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 59, 835, 64)   0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 59, 835, 128)  73856       max_pooling2d_1[0][0]            
___________________________________________________________________________________________

## Fit the model

In [19]:
tr_labs.shape

(9044, 107)

In [18]:
shp1 = np.ones(shape=(len(tr_imgs), 1)) * tr_imgs.shape[2]
shp2 = np.ones(shape=(len(val_imgs), 1)) * val_imgs.shape[2]
checkpointer = ModelCheckpoint(filepath="results/"+save_model_file, verbose=0)
hist = model.fit(x = [tr_imgs, tr_labs, shp1, tr_label_len],
                 y = tr_labs, batch_size = 32, epochs = n_epochs,
                 verbose = 1, callbacks = [checkpointer])
#                  validation_data = ([val_imgs, val_labs, shp2, shp2], val_labs))

Epoch 1/1


InvalidArgumentError: sequence_length(0) <= 417
	 [[Node: ctc/CTCLoss = CTCLoss[ctc_merge_repeated=true, ignore_longer_outputs_than_inputs=false, preprocess_collapse_repeated=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ctc/Log, ctc/ToInt64, ctc/ToInt32_2, ctc/ToInt32_1)]]

Caused by op 'ctc/CTCLoss', defined at:
  File "C:\Users\danny\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\danny\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\danny\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\danny\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\danny\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\danny\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py